# Text document Classification (Dataset)

https://www.kaggle.com/datasets/jensenbaxter/10dataset-text-document-classification

*   10 clases
*   1000 muestras de entrenamiento

In [ ]:
import zipfile
with zipfile.ZipFile('/content/Dataset_text.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Dataset_text/')

In [ ]:
from tensorflow import keras
batch_size = 32

In [ ]:
"""
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(2023).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)

In [ ]:
train_ds = keras.utils.text_dataset_from_directory(
    "/content/Dataset_text",
    batch_size=batch_size,
    validation_split=0.2,
    label_mode="categorical",
    seed = 2023,
    subset="training",
)

Found 1000 files belonging to 10 classes.
Using 800 files for training.


In [ ]:
val_ds = keras.utils.text_dataset_from_directory(
    "/content/Dataset_text",
    batch_size=batch_size,
    validation_split=0.2,
    label_mode="categorical",
    seed = 2023,
    subset="validation",
)

Found 1000 files belonging to 10 classes.
Using 200 files for validation.


In [ ]:
"""
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=batch_size
)
"""

'\ntest_ds = keras.utils.text_dataset_from_directory(\n    "aclImdb/test",\n    batch_size=batch_size\n)\n'

In [ ]:
# Visualizar características del primer lote (batch)
for inputs, targets in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32, 10)
targets.dtype: <dtype: 'float32'>
inputs[0]: tf.Tensor(b'Toward Armistice\r\nBy the fall of 1918, the Central Powers were unraveling on all fronts.\r\n\r\nDespite the Turkish victory at Gallipoli, later defeats by invading forces and an Arab revolt had combined to destroy the Ottoman economy and devastate its land, and the Turks signed a treaty with the Allies in late October 1918.\r\n\r\nAustria-Hungary, dissolving from within due to growing nationalist movements among its diverse population, reached an armistice on November 4. Facing dwindling resources on the battlefield, discontent on the homefront and the surrender of its allies, Germany was finally forced to seek an armistice on November 11, 1918, ending World War I.', shape=(), dtype=string)
targets[0]: tf.Tensor([0. 0. 0. 0. 1. 0. 0. 0. 0. 0.], shape=(10,), dtype=float32)


# Data preparation



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# No se utilizan ngrams (none)
max_length = 600
max_tokens = 20000
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,    # Limitar la entrada a 600 palabras
)

# Extraer solo el texto (features) de los datos de entrada para calcular el vocabulario
text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorization.adapt(text_only_train_ds)

"""
int_train_ds = train_ds.map(
    lambda x, y: (tf.one_hot(text_vectorization(x), max_tokens), y),
    num_parallel_calls=4)

int_val_ds = val_ds.map(
    lambda x, y: (tf.one_hot(text_vectorization(x), max_tokens), y),
    num_parallel_calls=4)

int_test_ds = test_ds.map(
    lambda x, y: (tf.one_hot(text_vectorization(x), max_tokens), y),
    num_parallel_calls=4)
"""



int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

"""
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
"""

'\nint_test_ds = test_ds.map(\n    lambda x, y: (text_vectorization(x), y),\n    num_parallel_calls=4)\n'

# Modelo

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input, Dropout, Bidirectional

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(10, activation="softmax")(x)
model_Embedded2 = keras.Model(inputs, outputs)

model_Embedded2.compile(optimizer="rmsprop",
                        loss="categorical_crossentropy",
                        metrics=["accuracy"])

model_Embedded2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 5194634 (19.82 MB)
Trainable params: 5194634 (19.82 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

# Entrenamiento

In [ ]:
# Utilización de caché
model_Embedded2.fit(
    int_train_ds,
    validation_data=int_val_ds,
    epochs=20,
    #callbacks=callbacks
    )

Epoch 1/20
25/25 [==============================] - 22s 340ms/step - loss: 2.2809 - accuracy: 0.2163 - val_loss: 2.2354 - val_accuracy: 0.4150
Epoch 2/20
25/25 [==============================] - 5s 215ms/step - loss: 2.0758 - accuracy: 0.3900 - val_loss: 1.8596 - val_accuracy: 0.3950
Epoch 3/20
25/25 [==============================] - 6s 249ms/step - loss: 1.7437 - accuracy: 0.3862 - val_loss: 1.8433 - val_accuracy: 0.3750
Epoch 4/20
25/25 [==============================] - 6s 238ms/step - loss: 1.5502 - accuracy: 0.4263 - val_loss: 1.5880 - val_accuracy: 0.4600
Epoch 5/20
25/25 [==============================] - 5s 193ms/step - loss: 1.3732 - accuracy: 0.5188 - val_loss: 1.5211 - val_accuracy: 0.5000
Epoch 6/20
25/25 [==============================] - 6s 251ms/step - loss: 1.2145 - accuracy: 0.5663 - val_loss: 1.3038 - val_accuracy: 0.6000
Epoch 7/20
25/25 [==============================] - 6s 250ms/step - loss: 1.0389 - accuracy: 0.6712 - val_loss: 1.1732 - val_accuracy: 0.6600
Epoch

#Evaluación del modelo

Sobre datos test